In [2]:
import os
dir='/kaggle/input/ucf101/UCF101/UCF-101'
videos=os.listdir(os.path.join(dir,'BabyCrawling'))

In [3]:
import cv2
from google.colab.patches import cv2_imshow
for video in videos:
  video_path = os.path.join(dir,'BabyCrawling',video)
  videoFrames = cv2.VideoCapture(video_path)
  frame_count=0
  while True:
    success,frame = videoFrames.read()
    frame_count+=1
    if not success:
      break

  print(frame_count)

191
128
118
171
182
106
181
96
113
149
146
222
107
199
110
127
180
242
195
142
114
165
109
153
100
152
211
97
121
180
249
262
199
272
259
157
160
190
180
218
236
119
186
114
125
216
137
117
220
208
224
294
175
129
243
228
208
177
181
198
192
166
202
110
112
172
118
142
196
168
111
102
165
207
93
117
112
140
168
143
199
330
161
120
216
223
159
205
206
223
205
251
160
105
143
264
109
118
183
104
101
113
185
179
95
85
166
173
160
175
231
172
146
214
179
222
125
110
117
123
86
105
142
159
198
194
145
145
204
199
112
104


In [4]:
import cv2
import os
import numpy as np

def extract_clips_from_video(video_path, clip_len=16):
    """
    Takes a single video and returns a list of (clip, label)
    where each clip = 16 frames (112x112x3).
    """
    clips = []
    cap = cv2.VideoCapture(video_path)

    clip = []
    frame_count = 0

    while True:
        success, frame = cap.read()
        if not success:
            # save leftover frames if any
            if len(clip) > 0:
                while len(clip) < clip_len:
                    clip.append(clip[-1])       # pad last frame
                clips.append(np.array(clip))
            break

        # BGR → RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Resize to 112x112
        frame = cv2.resize(frame, (112,112))

        # Normalize
        frame = frame.astype("float32") / 255.0

        clip.append(frame)
        frame_count += 1

        # We got 16 frames → save as clip
        if frame_count == clip_len:
            clips.append(np.array(clip))
            clip = []
            frame_count = 0

    cap.release()
    return clips


In [7]:
# os.rmdir('/kaggle/working/videos')

FileNotFoundError: [Errno 2] No such file or directory: 'videos'

In [8]:
os.makedirs('videos', mode=0o777, exist_ok=False)

In [9]:
clips_folder = '/kaggle/working/videos'
video_folder = '/kaggle/input/ucf101/UCF101/UCF-101'
from tqdm import tqdm
for cls in tqdm(os.listdir(video_folder)[0:2], "Working on folder"):
    cls_path = os.path.join(video_folder, cls)
    for video in tqdm(os.listdir(cls_path),'Workin on video'):
        video_path = os.path.join(cls_path, video)

        save_path = os.path.join(clips_folder, cls, video)
        os.makedirs(save_path, exist_ok=True)

        clips = extract_clips_from_video(video_path)

        for i, single_clip in enumerate(clips):
            np.save(os.path.join(save_path, f'clip_{i}.npy'), single_clip)


Working on folder: 100%|██████████| 2/2 [00:27<00:00, 13.94s/it]


In [10]:
import os
import numpy as np

# Root folder with subfolders for each class

root_dir = "/kaggle/working/videos"

file_paths = []
labels = []


class_names = sorted(os.listdir(root_dir))
class_to_idx = {name: idx for idx, name in enumerate(class_names)}
print(class_names)


for class_name in class_names:
  class_folder = os.path.join(root_dir, class_name)
  for file_name in os.listdir(class_folder):
    clips=os.path.join(root_dir,class_name,file_name)
    for video in os.listdir(clips):

      if video.endswith(".npy"):
        file_paths.append(os.path.join(root_dir,class_name,file_name,video))
        labels.append(class_to_idx[class_name])

file_paths = np.array(file_paths)
labels = np.array(labels)

print("Total clips:", len(file_paths))
print("First 5 files:", file_paths[:5])
print("First 5 labels:", labels[:5])


['HorseRace', 'StillRings']
Total clips: 3397
First 5 files: ['/kaggle/working/videos/HorseRace/v_HorseRace_g03_c01.avi/clip_4.npy'
 '/kaggle/working/videos/HorseRace/v_HorseRace_g03_c01.avi/clip_8.npy'
 '/kaggle/working/videos/HorseRace/v_HorseRace_g03_c01.avi/clip_1.npy'
 '/kaggle/working/videos/HorseRace/v_HorseRace_g03_c01.avi/clip_10.npy'
 '/kaggle/working/videos/HorseRace/v_HorseRace_g03_c01.avi/clip_0.npy']
First 5 labels: [0 0 0 0 0]


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     file_paths, labels, test_size=0.33, random_state=42)

In [12]:
import tensorflow as tf

batch_size = 8
num_classes = len(set(labels))


def clip_generator(file_paths, labels):
    for path, label in zip(file_paths, labels):
        clip = np.load(path)               
        yield clip, label


train_dataset = tf.data.Dataset.from_generator(
    lambda: clip_generator(X_train, y_train),
    output_signature=(
        tf.TensorSpec(shape=(16, 112, 112, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)
test_dataset = tf.data.Dataset.from_generator(
    lambda: clip_generator(X_test, y_test),
    output_signature=(
        tf.TensorSpec(shape=(16, 112, 112, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)
train_dataset = train_dataset.shuffle(buffer_size=len(X_train))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.shuffle(buffer_size=len(X_test))
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)




2025-11-20 15:49:59.706164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763653799.921549      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763653799.965899      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

I0000 00:00:1763653814.274138      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [13]:
for X_batch, y_batch in test_dataset.take(1):
    print(X_batch.shape, y_batch.shape)

(8, 16, 112, 112, 3) (8,)


In [14]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(16, 112, 112, 3)),

    tf.keras.layers.Conv3D(64, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling3D(pool_size=2, strides=2, padding='same'),

    tf.keras.layers.Conv3D(128, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling3D(pool_size=2, strides=2, padding='same'),

    tf.keras.layers.Conv3D(256, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.Conv3D(256, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling3D(pool_size=2, strides=2, padding='same'),

    tf.keras.layers.Conv3D(512, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.Conv3D(512, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling3D(pool_size=2, strides=2, padding='same'),

    tf.keras.layers.Conv3D(512, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.Conv3D(512, kernel_size=3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling3D(pool_size=2, strides=2, padding='same'),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.GlobalAveragePooling3D(),

    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [15]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 16, 112, 112,   │         5,248 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 8, 56, 56, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 8, 56, 56, 128) │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 4, 28, 28, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 4, 28, 28, 256) │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 4, 28, 28, 256) │     1,769,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 2, 14, 14, 256) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (None, 2, 14, 14, 512) │     3,539,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_5 (Conv3D)               │ (None, 2, 14, 14, 512) │     7,078,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 1, 7, 7, 512)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_6 (Conv3D)               │ (None, 1, 7, 7, 512)   │     7,078,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_7 (Conv3D)               │ (None, 1, 7, 7, 512)   │     7,078,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_4 (MaxPooling3D)  │ (None, 1, 4, 4, 512)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 4, 4, 512)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling3d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling3D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │     2,101,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │         8,194 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,546,690 (177.56 MB)

 Trainable params: 46,546,690 (177.56 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "c3d_best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

In [ ]:



epochs=10
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/10


I0000 00:00:1763653828.152100    1582 service.cc:148] XLA service 0x7cfbe800a1a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763653828.152799    1582 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1763653828.668951    1582 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1763653838.142085    1582 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    285/Unknown 82s 220ms/step - accuracy: 0.5936 - loss: 0.6649

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


285/285 ━━━━━━━━━━━━━━━━━━━━ 99s 281ms/step - accuracy: 0.5936 - loss: 0.6650 - val_accuracy: 0.6194 - val_loss: 0.5362
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.6410 - loss: 0.5571

285/285 ━━━━━━━━━━━━━━━━━━━━ 80s 253ms/step - accuracy: 0.6412 - loss: 0.5570 - val_accuracy: 0.9492 - val_loss: 0.3169
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.9373 - loss: 0.2192

285/285 ━━━━━━━━━━━━━━━━━━━━ 79s 253ms/step - accuracy: 0.9373 - loss: 0.2190 - val_accuracy: 0.9599 - val_loss: 0.1242
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.9533 - loss: 0.1432

285/285 ━━━━━━━━━━━━━━━━━━━━ 78s 254ms/step - accuracy: 0.9533 - loss: 0.1431 - val_accuracy: 0.9840 - val_loss: 0.0510
Epoch 5/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.9553 - loss: 0.1243

285/285 ━━━━━━━━━━━━━━━━━━━━ 79s 254ms/step - accuracy: 0.9553 - loss: 0.1242 - val_accuracy: 0.9920 - val_loss: 0.0267
Epoch 6/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.9911 - loss: 0.0396

285/285 ━━━━━━━━━━━━━━━━━━━━ 79s 254ms/step - accuracy: 0.9911 - loss: 0.0396 - val_accuracy: 0.9938 - val_loss: 0.0169
Epoch 7/10
238/285 ━━━━━━━━━━━━━━━━━━━━ 9s 199ms/step - accuracy: 0.9882 - loss: 0.0374

In [ ]:
test = '/kaggle/input/tstestt/videoplayback.mp4'
clips=extract_clips_from_video(test)

In [ ]:
model.predict(clips)

In [ ]:
clips_expanded = clips.expand_dims(clips,axis=0)

In [ ]:
model.predict(clips)